In [1]:
import qiskit
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit_aer.noise import depolarizing_error, coherent_unitary_error, NoiseModel
from qiskit.quantum_info import SparsePauliOp

from qiskit.primitives import BackendEstimator

from zne import zne, ZNEStrategy
from zne.noise_amplification import *
from zne.extrapolation import *

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# We want more physical error model here, over-rotation on the target in CNOT. And do some test before doing it.

def coherent_noise_backend(
        noise_strength: float = 0.1,
        Simulator = AerSimulator
):
    # Create the coherent error channel on 2Q gates
    # Copy-pasted from `test_ampcal1Q` in https://programtalk.com/python-more-examples/qiskit.providers.aer.noise.NoiseModel/
    err_unitary = np.matrix(
        [[1, 0, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 1, 0, 0]],
        dtype=complex
    )
    err_coherent = coherent_unitary_error(err_unitary)

    # Create a noise model from the error channels
    noisemodel = NoiseModel()
    noisemodel.add_all_qubit_quantum_error(err_coherent, ['cx'])

    backend = Simulator(noise_model=noisemodel)
    return backend

In [24]:
backend = coherent_noise_backend()
# backend = AerSimulator()

qc = QuantumCircuit(2)
qc.x(0)
qc.cnot(0, 1)
qc.measure_all()

qc_trans = transpile(qc, backend=backend, optimization_level=3)
counts = backend.run(qc_trans).result().get_counts()
# sns.histplot(counts)
print(counts)

{'01': 1024}
